In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("MostPopularSuperhero3")
sc = SparkContext(conf = conf)

In [3]:
def extract_friends(super_hero_row):
    super_hero=super_hero_row.split()
    super_hero_id=int(super_hero[0])
    super_hero_friends=[int(sh_id) for sh_id in super_hero[1:]]
    return super_hero_id,super_hero_friends

superhero_friends_raw=sc.textFile("/home/ggomarr/Documents/Education/Udemy_Spark/23_Marvel-Graph.txt")
superhero_friends_count=(superhero_friends_raw.map(extract_friends)
                                              .map(lambda x: (x[0],len(x[1])))
                                              .reduceByKey(lambda x, y: x + y))

In [4]:
for superhero in superhero_friends_count.take(10):
    print('{:4d} - {:4d}'.format(superhero[0],superhero[1]))

   1 -    5
   2 -  122
   3 -   72
   4 -   14
   5 -    8
   6 -   21
   7 -    8
   8 -   15
   9 -   11
  10 -  187


In [7]:
def process_superhero_row(superhero_row):
    import re
    superhero=superhero_row.split(' ',1)
    superhero_id=int(superhero[0])
    superhero_name=re.sub('^[^a-zA-Z0-9\s]*|[^a-zA-Z0-9\s]*$','',superhero[1].strip())
    return (superhero_id,superhero_name)

superhero_names_raw=sc.textFile("/home/ggomarr/Documents/Education/Udemy_Spark/23_Marvel-Names.txt")
superhero_names=superhero_names_raw.map(process_superhero_row)

In [8]:
for superhero in superhero_names.take(10):
    print('{:4d} - {}'.format(superhero[0],superhero[1]))

   1 - 24-HOUR MAN/EMMANUEL
   2 - 3-D MAN/CHARLES CHAN
   3 - 4-D MAN/MERCURIO
   4 - 8-BALL
   5 - A
   6 - A'YIN
   7 - ABBOTT, JACK
   8 - ABCISSA
   9 - ABEL
  10 - ABOMINATION/EMIL BLO


In [9]:
superhero_friends_count_with_names=superhero_friends_count.join(superhero_names)

In [10]:
for superhero in superhero_friends_count_with_names.take(10):
    print('{:4d} - {:3d} - {}'.format(superhero[0],superhero[1][0],superhero[1][1]))

   2 - 122 - 3-D MAN/CHARLES CHAN
   4 -  14 - 8-BALL
   6 -  21 - A'YIN
   8 -  15 - ABCISSA
  10 - 187 - ABOMINATION/EMIL BLO
  12 -   6 - ABOMINATRIX
  14 -  15 - ADAM 3,031
  16 - 275 - ABSORBING MAN/CARL C
  18 -  34 - ACBA
  20 -   5 - ACHILLES


In [12]:
superhero_friends_count_with_names_sorted=(superhero_friends_count_with_names
                                           .map(lambda x: (x[1][0],(x[0],x[1][1])))
                                           .sortByKey(False))

In [13]:
for superhero in superhero_friends_count_with_names_sorted.take(10):
    print('{:3d} - {:4d} - {}'.format(superhero[0],superhero[1][0],superhero[1][1]))

1933 -  859 - CAPTAIN AMERICA
1741 - 5306 - SPIDER-MAN/PETER PAR
1528 - 2664 - IRON MAN/TONY STARK 
1426 - 5716 - THING/BENJAMIN J. GR
1394 - 6306 - WOLVERINE/LOGAN 
1386 - 3805 - MR. FANTASTIC/REED R
1371 - 2557 - HUMAN TORCH/JOHNNY S
1345 - 4898 - SCARLET WITCH/WANDA 
1289 - 5736 - THOR/DR. DONALD BLAK
1280 -  403 - BEAST/HENRY &HANK& P
